In [31]:
import numpy as np
import pandas as pd
import string
import spacy
import re
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
import random
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
import os
import time

In [2]:
if tf.config.list_physical_devices('GPU'):
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
    tf.config.experimental.set_virtual_device_configuration(physical_devices[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])

# ETAPA DE CARREGAMENTO DO DATASET

In [3]:
data = pd.read_json('tcc1.json', encoding='utf-8')

# ETAPA DE PRÉ-PROCESSAMENTO

In [4]:
titulo_input = ['<start> ' + m + ' <end>' for m in data.título.tolist()]
noticia_input = ['<start> ' + m + ' <end>' for m in data.texto.tolist()]

# ETAPA DE TOKENIZAÇÃO

In [5]:
def token(texto, tam_max):
    tokens = tf.keras.preprocessing.text.Tokenizer(lower=True, filters='', num_words=2**16)
    tokens.fit_on_texts(texto)
    tensor = tokens.texts_to_sequences(texto)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=tam_max)
    return tensor, tokens

In [6]:
data_input_tokens, data_input = token(noticia_input, 600)
data_target_tokens, target_input = token(titulo_input, 20)

In [7]:
num_encoder_tokens = data_input_tokens.shape[1]
num_decoder_tokens = data_target_tokens.shape[1]

# Divisão dos dados

In [8]:
input_data_train, input_data_test, input_decoder_train, input_decoder_test = train_test_split(data_input_tokens, data_target_tokens, test_size=0.2)

# Variáveis de configuração da rede

In [48]:
buffer = len(input_data_train)
batch = 16
steps_por_epoca = len(input_data_train)//batch
embedding_dim = 200
units = 1024
vocab_size_input = len(data_input.word_index)+1
vocab_size_target = len(target_input.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_data_train, input_decoder_train)).shuffle(buffer)
dataset = dataset.batch(batch, drop_remainder=True)

# Criação do dataset

In [10]:
encoder_input_data = np.zeros(
    (len(noticia_input), num_encoder_tokens, vocab_size_input),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(noticia_input), num_decoder_tokens, vocab_size_target),
    dtype='float32')

print(encoder_input_data.shape)
print(decoder_input_data.shape)

In [11]:
for i, input_text in enumerate(input_data_train):
    for t, char in enumerate(input_text):
        #print(i, "->", t, "->", data_input_tokens[char])
        encoder_input_data[i, t, char] = 1.

In [12]:
for i, decoder_text in enumerate(input_decoder_train):
    for t, char in enumerate(decoder_text):
        decoder_input_data[i, t, char] = 1.

# Arquitetura da Rede

In [35]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, emb_dim, units, batch):
        super(Encoder, self).__init__()
        self.batch = batch
        self.units = units
        self.embedding = Embedding(vocab_size, emb_dim)
        self.lstm = LSTM(units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    
    def call(self, x, hidden):
        x. self.embedding(x)
        output, state = self.lstm(x, initial_state = hidden)
        return output, state
    def intializer_hidden_state(self):
        return tf.zeros((self.batch, self.units))

"encoder_inputs = Input(shape=(None, num_encoder_tokens))\nencoder = keras.Sequential([\n    layers.Embedding(vocab_size_input, embedding_dim),\n    layers.Dense(vocab_size_input, activation='relu')])\nencoder_outputs, state_h, state_c = encoder(encoder_inputs)\nencoder_states = [state_h, state_c]"

In [36]:
encoder = Encoder(vocab_size_input, embedding_dim, units, batch)

In [37]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    query_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1

    return context_vector, attention_weights

In [38]:
attention_layer = BahdanauAttention(10)

In [39]:
class Decoder(tf.keras.Model):
    def __iniit__(self, vocab_size, emb_dim, units, batch):
        super(Decoder, self).__init__()
        self.batch = batch
        self.units = units
        self.embedding = Embedding(vocab_size, emb_dim)
        self.lstm = LSTM(units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.attention = BahdanauAttention(self.units)
    
    def call(self, x, hidden, output):
        context_vector, attention_weights = self.attention(hidden, output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        return output, state, attention_weights

"decoder_inputs = Input(shape=(None, num_decoder_tokens))\ndecoder_lstm = Embedding(vocab_size_input, embedding_dim)\ndecoder_lstm = LSTM(units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')\ndecoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)\ndecoder_dense = Dense(num_decoder_tokens, activation='softmax')\ndecoder_outputs = decoder_dense(decoder_outputs)"

In [40]:
decoder = Decoder(vocab_size_target, embedding_dim, units, batch)

In [ ]:
def erro(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [42]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=tf.keras.optimizers.Adam(), encoder=encoder, decoder=decoder)

In [43]:
@tf.function
def treino(input_data, target, hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, hidden = encoder(input_data, hidden)
    dec_hidden = hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * batch, 1)

    for t in range(1, targ.shape[1]):
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += erro(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [50]:
epocas = 10
erro_acumulado = []
for e in range(epocas):
    inicio = time.time()
    hidden = encoder.intializer_hidden_state()
    erro_total = 0
    for (batch, (input_data, target)) in enumerate(dataset.take(steps_por_epoca)):
        batch_loss = treino(input_data, target, hidden)
        erro_total += batch_loss
        
    print('Epoca {} Erro {:.04f}'.format(epoca+1, erro_total/steps+por_epoca))
    erro_acumulado.append(epoca+1, erro_total/steps+por_epoca)
    print('Para uma época levou {}s'.format(time.time() - inicio))
checkpoint.save(file_prefix = checkpoint_prefix)

AttributeError: in converted code:

    <ipython-input-43-97a6f1c6cfba>:6 treino  *
        enc_output, hidden = encoder(input_data, hidden)
    C:\Users\renat\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:778 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    <ipython-input-35-c101d4d12e36>:10 call  *
        x. self.embedding(x)

    AttributeError: 'Tensor' object has no attribute 'self'


In [22]:
model = Model([encoder_inputs, decoder_inputs], decoder_inputs)

In [23]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, 20)          0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


# Compilação do modelo

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinamento do modelo

In [25]:
model.fit([encoder_input_data, decoder_input_data], decoder_input_data, batch_size=batch, epochs=100, validation_split=0.2)

ValueError: Error when checking input: expected input_3 to have shape (None, 600) but got array with shape (600, 21661)

In [ ]:
model.save('sumAbstrat.h5')